# Processing
Demonstration of Processing endpoints.

In [1]:
import utils.DemoClient as client
import jwt
import json

## Initialise

### Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [2]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "demo.eoepca.org"
platform_domain = "test." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

client_id: 0e78147e-86ff-4487-80ae-e1309f617402
Client state saved to file: state.json


### User
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [3]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="bob"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

[Request] GET => https://test.demo.eoepca.org/.well-known/uma2-configuration
<Response [200]>
token_endpoint: https://test.demo.eoepca.org/oxauth/restv1/token
[Request] POST => https://test.demo.eoepca.org/oxauth/restv1/token


## ADES
The ADES provides user-specific endpoints, using a URL path prefix.

In [9]:
# ADES URLs
ades_base_url = "http://ades.demo.eoepca.org" #"http://ades." + platform_domain
ades_wps_url = ades_base_url + "/" + USER_NAME + "/zoo"; print("ADES WPS endpoint:", ades_wps_url)
ades_proc_url = ades_base_url + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

ADES WPS endpoint: http://ades.demo.eoepca.org/bob/zoo
ADES API Processes endpoint: http://ades.demo.eoepca.org/bob/wps3


### ADES API Processes

In [10]:
ades_access_token = None
app_name = "s-expression-0_0_2"

#### ADES: List Processes

In [11]:
# API Processes - List Processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes
[Process List] = 200 (OK)
Processes: ['nhi-0_0_3', 'DeployProcess', 'GetStatus', 'burned-area-1_0', 'burned-area-0_5', 'snuggs-0_3_0', 'UndeployProcess']


#### ADES: Deploy Application

In [12]:
# API Processes - Deploy Application
#response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-atom.json", id_token=user_id_token, access_token=ades_access_token)

response, access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-cwl-S3.json", id_token=user_id_token, access_token=ades_access_token)


Application details read from file: ../data/app-deploy-body-cwl-S3.json
[Request] POST => http://ades.demo.eoepca.org/bob/wps3/processes
[Deploy Response] = 201 (Created)


In [13]:
# Check deploy - list processes again
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes
[Process List] = 200 (OK)
Processes: ['nhi-0_0_3', 'DeployProcess', 'GetStatus', 'burned-area-1_0', 'burned-area-0_5', 'snuggs-0_3_0', 'UndeployProcess', 's-expression-0_0_2']


#### ADES: Get Application Details

In [14]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes/s-expression-0_0_2
[App Details] = 200 (OK)
{
  "process": {
    "id": "s-expression-0_0_2",
    "title": "s expressions",
    "abstract": "Applies s expressions to EO acquisitions",
    "version": "0.0.2",
    "jobControlOptions": [
      "sync-execute",
      "async-execute",
      "dismiss"
    ],
    "outputTransmission": [
      "value",
      "reference"
    ],
    "links": [
      {
        "rel": "canonical",
        "type": "application/json",
        "title": "Execute End Point",
        "href": "/watchjob/processes/s-expression-0_0_2/jobs/"
      }
    ],
    "inputs": [
      {
        "id": "input_reference",
        "title": "Input product reference",
        "abstract": "Input product reference",
        "minOccurs": "1",
        "maxOccurs": "1",
        "input": {
          "literalDataDomains": [
            {
              "dataType": {
                "name": "string"
              },
              "val

#### ADES: Execute Application

In [15]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/app-execute-body.json", id_token=user_id_token, access_token=ades_access_token)

Application execute details read from file: ../data/app-execute-body.json
[Request] POST => http://ades.demo.eoepca.org/bob/wps3/processes/s-expression-0_0_2/jobs
[Execute Response] = 201 (Created) => job=/bob/watchjob/processes/s-expression-0_0_2/jobs/3e8af1c4-a060-11ec-bb8a-8a8147a03f5b


#### ADES: Job Status

In [16]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

[Request] GET => http://ades.demo.eoepca.org/bob/watchjob/processes/s-expression-0_0_2/jobs/3e8af1c4-a060-11ec-bb8a-8a8147a03f5b
[Job Status] = 200 (OK) => 10/03/2022 10:53:02 - running
{
  "jobID": "3e8af1c4-a060-11ec-bb8a-8a8147a03f5b",
  "started": "2022-03-10T10:52:56.824069Z",
  "status": "running",
  "message": "running",
  "progress": "0",
  "links": [
    {
      "href": "http://ades.demo.eoepca.org/bob/watchjob/processes/s-expression-0_0_2/jobs/3e8af1c4-a060-11ec-bb8a-8a8147a03f5b",
      "rel": "self",
      "type": "application/json",
      "title": "get Status"
    }
  ]
}


In [17]:
# API Processes - Job Status (keep polling for completion)
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)

[Job Status] = 200 (OK) => 10/03/2022 10:53:06 - running
[Job Status] = 200 (OK) => 10/03/2022 10:53:16 - running
[Job Status] = 200 (OK) => 10/03/2022 10:53:27 - running
[Job Status] = 200 (OK) => 10/03/2022 10:53:38 - running
[Job Status] = 200 (OK) => 10/03/2022 10:53:49 - running
[Job Status] = 200 (OK) => 10/03/2022 10:53:59 - running
[Job Status] = 200 (OK) => 10/03/2022 10:54:10 - running
[Job Status] = 200 (OK) => 10/03/2022 10:54:21 - running
[Job Status] = 200 (OK) => 10/03/2022 10:54:32 - running
[Job Status] = 200 (OK) => 10/03/2022 10:54:42 - running
[Job Status] = 200 (OK) => 10/03/2022 10:54:53 - running
[Job Status] = 200 (OK) => 10/03/2022 10:55:04 - running
[Job Status] = 200 (OK) => 10/03/2022 10:55:15 - running
[Job Status] = 200 (OK) => 10/03/2022 10:55:25 - running
[Job Status] = 200 (OK) => 10/03/2022 10:55:36 - running
[Job Status] = 200 (OK) => 10/03/2022 10:55:47 - running
[Job Status] = 200 (OK) => 10/03/2022 10:55:58 - running
[Job Status] = 200 (OK) => 10/0

In [18]:
print(response.status_code, response.headers, response.text)

200 {'Server': 'nginx/1.19.1', 'Date': 'Thu, 10 Mar 2022 11:03:53 GMT', 'Content-Type': 'application/json;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, PUT, POST, DELETE, PATCH, OPTIONS', 'Access-Control-Allow-Headers': 'DNT,X-CustomHeader,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Authorization', 'Set-Cookie': 'auth_rpt-ades=eyJraWQiOiJkYjVlMGM5Ni0yZGQwLTQyNjktYTA5OC02NWZkNzIyNjFjMGFfc2lnX3JzMjU2IiwidHlwIjoiSldUIiwiYWxnIjoiUlMyNTYifQ.eyJhdWQiOiI1MjhlNGI0OS0xZjZmLTRmZjctYmM2MS0zNmQwOGIwMGM2OWEiLCJwY3RfY2xhaW1zIjp7ImF1ZCI6IjBlNzgxNDdlLTg2ZmYtNDQ4Ny04MGFlLWUxMzA5ZjYxNzQwMiIsInN1YiI6ImYxMmMyNTkyLTAzMzItNDlmNC1hNGZiLTcwNjNiM2MyYTg4OSIsInVzZXJfbmFtZSI6ImJvYiIsImlzcyI6Imh0dHBzOi8vdGVzdC5kZW1vLmVvZXBjYS5vcmciLCJle

#### ADES: Job Result

In [19]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

[Request] GET => http://ades.demo.eoepca.org/bob/watchjob/processes/s-expression-0_0_2/jobs/3e8af1c4-a060-11ec-bb8a-8a8147a03f5b/result
[Job Result] = 200 (OK) => s3://ada6b03cde1846678c9ac78dfb4067c6:demo-user-bob/wf-3e8af1c4-a060-11ec-bb8a-8a8147a03f5b/catalog.json
{
  "outputs": [
    {
      "id": "wf_outputs",
      "time": "2022-03-10T11:03:50.811958Z",
      "value": {
        "inlineValue": "{\"StacCatalogUri\": \"s3://ada6b03cde1846678c9ac78dfb4067c6:demo-user-bob/wf-3e8af1c4-a060-11ec-bb8a-8a8147a03f5b/catalog.json\"}"
      }
    }
  ]
}


#### ADES: List Jobs

In [20]:
# API Processes - List Jobs
response, ades_access_token, job_ids = demo.proc_list_jobs(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print("Jobs:", job_ids)

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes/s-expression-0_0_2/jobs
[Job List] = 200 (OK)
Jobs: ['3e8af1c4-a060-11ec-bb8a-8a8147a03f5b']


#### ADES: Undeploy Application

In [21]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)

[Request] DELETE => http://ades.demo.eoepca.org/bob/wps3/processes/s-expression-0_0_2
[Undeploy Response] = 200 (OK)


In [22]:
# Check undeploy - list processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes
[Process List] = 200 (OK)
Processes: ['nhi-0_0_3', 'DeployProcess', 'GetStatus', 'burned-area-1_0', 'burned-area-0_5', 'snuggs-0_3_0', 'UndeployProcess']


## Workspace

In [ ]:
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None

### Workspace: Get Details

In [ ]:
# Workspace - Get Details
workspace_name = "rm-user-" + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()
print(json.dumps(workspace_details, indent = 2))

### Inspect S3 Bucket

In [ ]:
# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]

In [ ]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(' ->', obj)